In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import urllib 
import io

# 1.DEFINICIÓN
# Definimos la web que vamos a analizar y la principal
url = 'https://retina.elpais.com/tag/iniciativa_empresarial/a/'
url_main = 'https://retina.elpais.com/'

# Definimos tuplas para cada columna del csv que vamos a crear.
title = []
link = []
date = []
section = []
autor = []

# 2.CHEQUEAMOS ROBOTS.TXT Y CAMBIAMOS USER AGENT
def check_robots(url_main):
    req = urllib.urlopen(url_main + "robots.txt", data = None)
    print req.read()
    return req.read()

check_robots(url_main)

# Definimos una cabecera con el user agent que obtenemos al ejecutar manualmente. 
# Lo utilizaremos como parámetro al llamar a las url.
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}

# 3.PAGINADO
# Buscamos el botón ">" para ver el número de páginas que tiene la sección.Está ubicado en:
# <li class="paginacion-siguiente activo"><a href="XXXXXXX">Siguiente ›</a></li
res_pag = requests.get(url, headers=headers)
soup_pag = BeautifulSoup(res_pag.text, 'lxml')
link_pag = soup_pag.find('li', {'class': 'paginacion-siguiente activo'}).a['href'] 

# Al final de la cadena del link está el número de páginas que tiene la sección
num_pag = int(link_pag.split('/')[-1])

# 4.BÚSQUEDA
# Recorremos cada una de las páginas 
index = range(num_pag+1)
for number in index:
    
    # Llamamos a cada uno de las páginas.
    res = requests.get(url+str(number), headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
  
    # Buscamos el título de cada artículo y el link
    for i in soup.find_all('h2', {'class': 'articulo-titulo'}):
        
        # Título
        new_title = i.get_text()
        title.append(new_title)
        
        # Link al artículo
        article_link = i.a['href']
        new_link = "https://retina.elpais.com"+article_link
        link.append(new_link)
        
        # Dentro del link buscamos la fecha, el autor y la sección
        res_link = requests.get(new_link, headers=headers)
        soup_link = BeautifulSoup(res_link.text, 'lxml')
        
        # Fecha
        new_date = soup_link.time['datetime']
        date.append(new_date)
        
        # Autor (controlamos si no existe)
        new_autor = soup_link.find('div', {'class':'firma'})
        if not new_autor:
            new_autor = 'No autor'
        else:
            new_autor = new_autor.a.get_text()
        autor.append(new_autor.replace("\n", ""))
        
        # Sección (controlamos si no existe)
        new_section = soup_link.find('div', {'class':'articulo-antetitulo'})
        if not new_section:
            new_section = 'no section'
        else:
            new_section = new_section.get_text()
        section.append(new_section.replace("\n", ""))

# 5.GENERAR CSV
# Definimos un dataframe con la información extraida de la web
data = {'Title':title, 
        'PageLink':link,
        'Date':date,
        'Section':section,
        'Author':autor
        }
news = pd.DataFrame(data=data)
cols = ['Title', 'PageLink', 'Date', 'Section', 'Author']
news = news[cols]

# Generamos el csv retina_news.csv
filename = 'scraping_news.csv'
news.to_csv(filename, index=False, encoding='utf-8-sig', header=True, sep='|')

User-Agent: *
Disallow: /buscador/
Disallow: /pruebas/
Disallow: /publicidad/
Disallow: /notificarelacionadas
Disallow: /*.swf$
Disallow: /eskupTSUpdate
Disallow: /.well-known/amphtml/

